### Website Carbon scraper

This script takes a list of URL's in the urls.txt file, cleans the URL's, and scrapes the websitecarbon.com page.
It then extracts the grams of carbon value from the HTML, and saves the results to the carbondata.csv file.

I started by attempting to use MS Excel's cell formulas =WEBSERVICE() and =FILTERXML() functions, but these only work on windows. I then decided to try learning Julia, but decided it would be quicker to just use python and command line. The first version of this python notebook used wget to scrape the URL's, but in the end I needed to use requests.sessions and post the URL to the Websitecarbon.com webform to reliably retrieve the carbon data.

To rerun this script, check the list of URL's in the urls.txt file are up to date, clear out any cached HTML files in the ./html directory, and run all cells.

In [ ]:
import re
import os
import csv
import requests
import pandas as pd

In [ ]:
urls = open('urls.txt').readlines()
urls = [u.rstrip('\n').strip().rstrip('/').replace('https://', '').replace('http://', '').replace('www.', '') for u in urls]
urls = [u for u in urls if u]  # remove empties

In [ ]:
def scrapeWebCarbon(url):
    '''
    Function takes a URL and scrapes the html to the disk, then extracts the grams of carbon from the HTML header.
    Returns a list
    '''
    carbonurl = 'https://www.websitecarbon.com'
    cleanurl = url.replace('.', '-').replace('/', '-')
    headers = {'User-Agent': 'Mozilla/5.0'}
    payload = {'wgd-cc-url':url,
               'wgd-cc-retest': 'true'}
    
    directory = 'html'
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Make the request and save the response
    scrapefile = "./html/" + cleanurl + ".html"
    if not os.path.exists(scrapefile) or os.path.getsize(scrapefile) == 0:
        print('scraping ' + url)
        session = requests.Session()
        sess = session.post(carbonurl, headers=headers, data=payload)
        with open(scrapefile, "w") as f:
            f.write(sess.text)
            f.close()
    else:
        print('using cached html for ' + url)
        
    # Load the cached response file and search for the data
    grams = !grep -E 'grams": (.*),$' ./html/{cleanurl}.html | cut -d: -f2 | cut -d, -f1
    if len(grams) > 0:
        grams = grams[0].strip()
    
    return [url, 'grams of CO2', grams]

In [ ]:
carbondata = [scrapeWebCarbon(u) for u in urls]

In [ ]:
with open('carbondata.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['url', 'type', 'value'])
    writer.writerows(carbondata)

Now you can multiply these values by your website pageviews analytics to calculate your website's carbon footprint.

In [ ]:
# Take a look at the data
carbondata = pd.read_csv('carbondata.csv')
carbondata